In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline   
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
import joblib
import os


# Business Understanding

## Statement

One of the largest retail chains in the world wants to use their vast data source to build an efficient forecasting model to predict the sales for each SKU in its portfolio at its 76 different stores using historical sales data for the past 3 years on a week-on-week basis. Sales and promotional information is also available for each week - product and store wise.

However, no other information regarding stores and products are available. Can you still forecast accurately the sales values for every such product/SKU-store combination for the next 12 weeks accurately? If yes, then dive right in!

## Data Dictionary

| Field Name        | Data Type / Format                     | Example | Description                                                                                         | Allowed / Expected Range          |
|-------------------|----------------------------------------|---------|-----------------------------------------------------------------------------------------------------|-----------------------------------|
| `record_ID`       | `int64` (sequential key)               | `1`     | Surrogate key, unique for each row. No business meaning.                                            | ≥ 1, unique                        |
| `week`            | `string` date in **YY/MM/DD** (e.g. 17/01/11) | `17/01/11` | Calendar week identifier; usually week‑ending date. Convert to `datetime` for analysis.             | Valid dates                        |
| `store_id`        | `int32`                                | `8091`  | Identifier of the retail outlet. Links to **Store** dimension.                                      | Positive integers                  |
| `sku_id`          | `int32`                                | `216418`| Identifier for the Stock‑Keeping Unit. Links to **Product/SKU** dimension.                          | Positive integers                  |
| `total_price`     | `float64` (currency)                   | `99.0375`| Actual net revenue for the SKU‑store‑week (after discounts).                                        | ≥ 0                                |
| `base_price`      | `float64` (currency)                   | `111.8625`| Regular (list) price for the SKU in that week.                                                      | ≥ 0                                |
| `is_featured_sku` | `int8` (binary flag) → `bool`          | `0`     | Was the SKU advertised in a flyer/e‑mail feature? `1` = Yes, `0` = No.                              | {0, 1}                             |
| `is_display_sku`  | `int8` (binary flag) → `bool`          | `0`     | Did the SKU have an in‑store display? `1` = Yes, `0` = No.                                          | {0, 1}                             |
| `units_sold`      | `int64`                                | `20`    | Quantity sold for the SKU‑store‑week.                                                               | ≥ 0                                |

# EDA

In [18]:
df = pd.read_csv("../data/raw/train.csv")
df.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20
1,2,17/01/11,8091,216419,99.0375,99.0375,0,0,28
2,3,17/01/11,8091,216425,133.9500,133.9500,0,0,19
3,4,17/01/11,8091,216233,133.9500,133.9500,0,0,44
4,5,17/01/11,8091,217390,141.0750,141.0750,0,0,52


In [19]:
df.describe()

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
count,150150.000000,150150.000000,150150.000000,150149.000000,150150.000000,150150.000000,150150.000000,150150.000000
mean,106271.555504,9199.422511,254761.132468,206.626751,219.425927,0.095611,0.133200,51.674206
std,61386.037861,615.591445,85547.306447,103.308516,110.961712,0.294058,0.339792,60.207904
min,1.000000,8023.000000,216233.000000,41.325000,61.275000,0.000000,0.000000,1.000000
25%,53111.250000,8562.000000,217217.000000,130.387500,133.237500,0.000000,0.000000,20.000000
50%,106226.500000,9371.000000,222087.000000,198.075000,205.912500,0.000000,0.000000,35.000000
75%,159452.750000,9731.000000,245338.000000,233.700000,234.412500,0.000000,0.000000,62.000000
max,212644.000000,9984.000000,679023.000000,562.162500,562.162500,1.000000,1.000000,2876.000000


# Data Preprocessing

In [20]:
# df['week'] = pd.to_datetime(df['week'], format='%y/%m/%d')

# # Sort by keys to ensure correct lag computation
# df = df.sort_values(['store_id', 'sku_id', 'week'])

# # Create 12 lag features of units_sold for each SKU‑store combo
# for lag in range(1, 13):
#     df[f'units_sold_lag_{lag}'] = df.groupby(['store_id', 'sku_id'])['units_sold'].shift(lag)

# # Drop rows that don't have full lag history
# df_model = df.dropna().copy()

# # Feature matrix and target vector
# feature_cols = (
#     ['store_id', 'sku_id', 'base_price', 'total_price',
#      'is_featured_sku', 'is_display_sku'] +
#     [f'units_sold_lag_{lag}' for lag in range(1, 13)]
# )
# X = df_model[feature_cols]
# y = df_model['units_sold']


In [21]:
class LagFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, n_lags=12, drop_train_na=True):
        self.n_lags = n_lags
        self.drop_train_na = drop_train_na   # drop only in training rows

    def fit(self, X, y=None):
        return self                       # nothing to learn

    def transform(self, X):
        df = X.copy()
        df = df.sort_values(['store_id', 'sku_id', 'week'])
        for lag in range(1, self.n_lags + 1):
            df[f'units_sold_lag_{lag}'] = (
                df.groupby(['store_id', 'sku_id'])['units_sold'].shift(lag)
            )
        return df

In [22]:
N_LAGS = 12
FEATURE_COLS = (
    ['store_id', 'sku_id', 'base_price', 'total_price',
     'is_featured_sku', 'is_display_sku'] +
    [f'units_sold_lag_{i}' for i in range(1, N_LAGS + 1)]
)

In [23]:
# # Store data to the preprocessed folder
# saved_data = pd.concat([X, y], axis=1)
# saved_data.to_csv("../data/processed/train.csv", index=False)

# Modeling

In [24]:
# Use the last 12 weeks of data as a hold‑out validation set
cutoff_date = df_model['week'].max() - pd.Timedelta(weeks=12)
train_mask = df_model['week'] < cutoff_date
X_train, X_val = X[train_mask], X[~train_mask]
y_train, y_val = y[train_mask], y[~train_mask]

In [25]:

rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42,
    oob_score=False
)

pipeline = Pipeline([
    ("lagger",  LagFeatureEngineer(n_lags=N_LAGS, drop_train_na=False)),
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("model",   rf_model)
])


In [28]:
DATA_DIR = "../data/raw/"
train = pd.read_csv(f"{DATA_DIR}train.csv")
train["set"] = "train"

try:
    test = pd.read_csv(f"{DATA_DIR}test.csv")        # supply your test file if you have one
    test["units_sold"] = pd.NA            # placeholder
    test["set"] = "test"
    full = pd.concat([train, test], ignore_index=True)
except FileNotFoundError:
    test = None
    full = train

# Ensure week is datetime
full["week"] = pd.to_datetime(full["week"], format="%y/%m/%d")

In [29]:
# ─────────────────────────────────────────────────────────────
# 3.  Apply lag transformer once
# ─────────────────────────────────────────────────────────────
lagged = pipeline.named_steps["lagger"].transform(full)

# Split back out
train_lagged = lagged[lagged["set"] == "train"]
if test is not None:
    test_lagged  = lagged[lagged["set"] == "test"]

# Drop rows without complete lag history in train only
train_lagged = train_lagged.dropna(subset=[f'units_sold_lag_{i}'
                                           for i in range(1, N_LAGS + 1)])

X_train = train_lagged[FEATURE_COLS]
y_train = train_lagged["units_sold"]

In [30]:
pipeline.named_steps["model"].fit(X_train, y_train)

RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)

In [35]:
last_12_weeks = (
    train_lagged["week"]
    .drop_duplicates()
    .sort_values()
    .tail(12)
)

val_mask = train_lagged["week"].isin(last_12_weeks)
train_mask = ~val_mask

X_train_final = train_lagged.loc[train_mask, FEATURE_COLS]
y_train_final = train_lagged.loc[train_mask, "units_sold"]

X_val = train_lagged.loc[val_mask, FEATURE_COLS]
y_val = train_lagged.loc[val_mask, "units_sold"]

# 7.2  Re‑fit the RF on *earlier* weeks only  (optional: reuse previous fit)
#pipeline.named_steps["model"].fit(X_train_final, y_train_final)

# 7.3  Predict and score
y_pred = pipeline.named_steps["model"].predict(X_val)

mae  = mean_absolute_error(y_val, y_pred)
#rmse = mean_squared_error(y_val, y_pred, squared=False)
mape = np.mean(np.abs((y_val - y_pred) / np.where(y_val == 0, 1, y_val))) * 100
#r2   = r2_score(y_val, y_pred)

print("\n── Validation Metrics (last‑12‑weeks) ──")
print(f"MAE :  {mae:,.2f}")
#print(f"RMSE:  {rmse:,.2f}")
print(f"MAPE:  {mape:.2f}%")
#print(f"R²  :  {r2:.4f}")


── Validation Metrics (last‑12‑weeks) ──
MAE :  5.12
MAPE:  18.27%


# Model Exporting

In [36]:
joblib.dump(pipeline, "sales_forecast_pipeline_rf.pkl")
print("Pipeline saved as  sales_forecast_pipeline_rf.pkl")

Pipeline saved as  sales_forecast_pipeline_rf.pkl


In [43]:
len(X_test.columns.to_list())

18

In [38]:
X_test

,store_id,sku_id,base_price,total_price,is_featured_sku,is_display_sku,units_sold_lag_1,units_sold_lag_2,units_sold_lag_3,units_sold_lag_4,units_sold_lag_5,units_sold_lag_6,units_sold_lag_7,units_sold_lag_8,units_sold_lag_9,units_sold_lag_10,units_sold_lag_11,units_sold_lag_12
162930,8023,216233,141.0750,141.0750,0,0,119,110,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158310,8023,216233,136.8000,136.8000,0,0,101,127,131,90,98,212,22,<NA>,119,110,82,NaN
153690,8023,216233,139.6500,118.2750,0,0,159,110,120,164,146,92,71,122,118,135,83,75
159465,8023,216233,138.2250,138.2250,0,0,106,127,78,100,171,128,110,186,102,133,124,93
154845,8023,216233,139.6500,139.6500,0,0,152,168,158,118,105,129,214,111,160,121,129,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157079,9984,679023,234.4125,234.4125,0,0,4,10,13,7,8,10,24,20,9,24,13,15
152459,9984,679023,234.4125,186.6750,0,0,22,18,10,9,9,15,5,11,39,17,14,36
162854,9984,679023,234.4125,234.4125,0,0,28,7,8,15,<NA>,22,18,10,9,9,15,5
158234,9984,679023,234.4125,191.6625,0,0,18,40,9,6,14,13,5,16,19,20,15,20


In [44]:
if test is not None:
    loaded_pipe = joblib.load("sales_forecast_pipeline_rf.pkl")
    # Re‑use already‑computed lagged DataFrame
    X_test = test_lagged[FEATURE_COLS]
    preds = loaded_pipe.predict(X_test)
    test_lagged["units_sold_pred"] = preds
    test_lagged.to_csv("test_with_predictions.csv", index=False)
    print("✓ Predictions written to test_with_predictions.csv")

KeyError: 'week'